In [238]:
import requests
from bs4 import BeautifulSoup
import re
import json

In [239]:
URL = "https://quotes.toscrape.com/"

def get_texts():
    html_doc = requests.get(URL)
    soup = BeautifulSoup(html_doc.text, "html.parser")
    text_list = []

    content = soup.select("div[class=row] div[class=col-md-8] div[class=quote] span[class=text]")
    for el in content:
        el = str(el)
        reg = re.search(r'“\D+”', el)
        if reg == None:
            text_list.append(el[35:-7])
        else:
            text_list.append(reg.group())

    return text_list


def get_tags():
    html_doc = requests.get(URL)
    soup = BeautifulSoup(html_doc.text, "html.parser")
    tags_list = []

    content = soup.select("div[class=row] div[class=col-md-8] div[class=quote] div[class=tags]")
    for el in content:
        tags_for_one = []
        for i in el:
            i = str(i)
            if 'class="tag"' in i:
                tags_for_one.append(re.search(r">\D+<", i).group()[1:-1])
        tags_list.append(tags_for_one)
    return tags_list


def get_autors():
    html_doc = requests.get(URL)
    soup = BeautifulSoup(html_doc.text, "html.parser")
    autors_list = []

    content = soup.select("div[class=row] div[class=col-md-8] div[class=quote] small[class=author]")
    for el in content:
        el = str(el)
        autors_list.append(re.search(r">\D+<", el).group()[1:-1])

    return autors_list

In [212]:
all = []

index = 0
for quote in get_texts():
    dict_with_info = dict()
    dict_with_info["tags"] = get_tags()[index]
    dict_with_info["author"] = get_autors()[index]
    dict_with_info["quote"] = quote
    all.append(dict_with_info)
    index += 1

In [213]:
with open("qoutes.json", "w", encoding="utf=8") as f:
    json.dump(all, f, ensure_ascii=False, indent=4)

In [240]:
def get_url_list():
    html_doc = requests.get(URL)
    soup = BeautifulSoup(html_doc.text, "html.parser")  
    url_list = []

    content = soup.select("div[class=container] div[class=row] div[class=col-md-8] div[class=quote] span a") 
    for link in content:
        reg = re.search(r"\D+", link['href']).group()   
        url_list.append(reg)

    return url_list
    

In [241]:
all = []

for link in get_url_list():
    html_doc = requests.get(URL + link)
    soup = BeautifulSoup(html_doc.text, "html.parser")
    dict_for_one = dict()

    content = soup.select("div[class=container] div[class=author-details]")
    for el in content:
        dict_for_one["fullname"] = el.find("h3", attrs={"class" : "author-title"}).text
        dict_for_one["born_date"] = el.find("span", attrs={"class" : "author-born-date"}).text
        dict_for_one["born_location"] = el.find("span", attrs={"class" : "author-born-location"}).text
        dict_for_one["description"] = str(el.find("div", attrs={"class" : "author-description"}).text).strip()
    
    all.append(dict_for_one)

In [243]:
with open("authors.json", "w", encoding="utf=8") as f:
    json.dump(all, f, ensure_ascii=False, indent=4)